# PGA Dataset
Courtesy of some folks at PGA

## 23 Tournaments, 6+ years of juicy golf data

```
	"Waste Management Phoenix Open": {
		"2010": {
			"course_info": "TPC Scottsdale",
			"data": [{
				"fin_text": "1",
				"player_name": "Mahan, Hunter",
				"player_num": 24781,
				"sg_app": 1.66175,
				"sg_arg": -0.7355,
				"sg_ott": 1.11975,
				"sg_putt": 0.977,
				"sg_t2g": 2.0469999999999997,
				"sg_total": 3.0239999999999996,
				"total": -16
			}, {
				"fin_text": "2",
				"player_name": "Fowler, Rickie",
				"player_num": 32102,
				"sg_app": 1.9255,
				"sg_arg": 0.2237499999999999,
				"sg_ott": 1.1275,
				"sg_putt": -0.50375,
				"sg_t2g": 3.2780000000000005,
				"sg_total": 2.7742500000000003,
				"total": -15
			}, ...
```
Context for the understanding the data: https://www.pgatour.com/news/2016/05/31/strokes-gained-defined.html

### Attribute Definitions
- fin_text : final ranking in tournament
- sg_app : strokes gained from approach shot
- sg_arg : strokes gained around the green
- sg_ott : strokes gained off the tee
- sg_putt : strokes gained for putting
- sg_t2g : ???
- sg_total : strokes gained total
- total : total score at the end of tournament

## Steps: 
1) Determine what our feature (x) values are

    - SG values are an easy pick for this

2) Determine what our label (y) value is 

    - final ranking or total score would both be acceptable here, but we will use total score for simplicity sake

3) Restructure the data to be easier to vectorize into feature data per sample

    - This demonstrates some of the ETL capability of Spark

4) Try Linear Regression as a baseline

    - Demonstrates the easy of using the pre-packaged ML Library in Spark

5) Check RMSE (Root Mean Squared Error) to evaluate performance

In [5]:
features = ["sg_app", "sg_arg", "sg_ott", "sg_putt", "sg_t2g", "sg_total"]
label = "total"
meta = "player_name"

In [27]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PGA") \
    .getOrCreate()


sc = spark.sparkContext

In [28]:
path = "./tournaments.json"
pgaDF = spark.read.json(path)

In [8]:
pgaDF.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



In [9]:
path = "./tournaments.json"
pgaDF = spark.read.json(path, multiLine=True)
# Spark will not always hold your hand. Don't panic. Read the docs
pgaDF.show()

+------------------------------------+--------------------------+----------------------+------------------------------+----------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+------------------------------+------------------------------------+--------------------+------------------------+--------------------+----------------------+---------------------------------+-----------------------+-----------------------------+--------------------+-----------------------------+-----------------------+
|A Military Tribute at The Greenbrier|Arnold Palmer Invitational|Barracuda Championship|Dell Technologies Championship|FedEx St. Jude Classic|Fort Worth Invitational|        Genesis Open|  John Deere Classic|    PGA Championship|        RBC Heritage|Sanderson Farms Championship|Sentry Tournament of Champions|Shriners Hospitals for Children Open| Sony Open in Hawaii|THE PLAYERS Championship|   TOU

In [10]:
pgaDF.printSchema()

root
 |-- A Military Tribute at The Greenbrier: struct (nullable = true)
 |    |-- 2010: struct (nullable = true)
 |    |    |-- course_info: string (nullable = true)
 |    |    |-- data: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- fin_text: string (nullable = true)
 |    |    |    |    |-- player_name: string (nullable = true)
 |    |    |    |    |-- player_num: long (nullable = true)
 |    |    |    |    |-- sg_app: double (nullable = true)
 |    |    |    |    |-- sg_arg: double (nullable = true)
 |    |    |    |    |-- sg_ott: double (nullable = true)
 |    |    |    |    |-- sg_putt: double (nullable = true)
 |    |    |    |    |-- sg_t2g: double (nullable = true)
 |    |    |    |    |-- sg_total: double (nullable = true)
 |    |    |    |    |-- total: long (nullable = true)
 |    |-- 2011: struct (nullable = true)
 |    |    |-- course_info: string (nullable = true)
 |    |    |-- data: array (nullable = true)
 |

In [11]:
# arnoldPalmerDF = pgaDF.withColumn("palmer")
# arnoldPalmerDF.printSchema()
apdf = pgaDF.select("Arnold Palmer Invitational")
apdf.printSchema()

root
 |-- Arnold Palmer Invitational: struct (nullable = true)
 |    |-- 2010: struct (nullable = true)
 |    |    |-- course_info: string (nullable = true)
 |    |    |-- data: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- fin_text: string (nullable = true)
 |    |    |    |    |-- player_name: string (nullable = true)
 |    |    |    |    |-- player_num: long (nullable = true)
 |    |    |    |    |-- sg_app: double (nullable = true)
 |    |    |    |    |-- sg_arg: double (nullable = true)
 |    |    |    |    |-- sg_ott: double (nullable = true)
 |    |    |    |    |-- sg_putt: double (nullable = true)
 |    |    |    |    |-- sg_t2g: double (nullable = true)
 |    |    |    |    |-- sg_total: double (nullable = true)
 |    |    |    |    |-- total: long (nullable = true)
 |    |-- 2011: struct (nullable = true)
 |    |    |-- course_info: string (nullable = true)
 |    |    |-- data: array (nullable = true)
 |    |    |

In [12]:
from pyspark.sql.functions import explode, array
allyearsdf = apdf.select(array("Arnold Palmer Invitational.2010.data", \
                           "Arnold Palmer Invitational.2011.data", \
                           "Arnold Palmer Invitational.2012.data", \
                           "Arnold Palmer Invitational.2013.data",
                           "Arnold Palmer Invitational.2014.data", \
                           "Arnold Palmer Invitational.2015.data", \
                           "Arnold Palmer Invitational.2016.data", \
                           "Arnold Palmer Invitational.2017.data", \
                           "Arnold Palmer Invitational.2018.data", \
                           "Arnold Palmer Invitational.2019.data").alias("statsdata"))
allyearsdf.show()

+--------------------+
|           statsdata|
+--------------------+
|[[[1, Els, Ernie,...|
+--------------------+



In [13]:
statscol = allyearsdf.select(explode(allyearsdf.statsdata))
statscol.show()
statscol.count()

+--------------------+
|                 col|
+--------------------+
|[[1, Els, Ernie, ...|
|[[1, Laird, Marti...|
|[[1, Woods, Tiger...|
|[[1, Woods, Tiger...|
|[[1, Every, Matt,...|
|[[1, Every, Matt,...|
|[[1, Day, Jason, ...|
|[[1, Leishman, Ma...|
|[[1, McIlroy, Ror...|
|[[1, Molinari, Fr...|
+--------------------+



10

In [14]:
allyearsstatscol = statscol.select(explode(statscol.col))
allyearsstatscol.show()
allyearsstatscol.count()

+--------------------+
|                 col|
+--------------------+
|[1, Els, Ernie, 6...|
|[T2, Molinari, Ed...|
|[T2, Na, Kevin, 2...|
|[T4, Couch, Chris...|
|[T4, Goosen, Reti...|
|[6, Curtis, Ben, ...|
|[T7, Hayes, J.P.,...|
|[T7, Imada, Ryuji...|
|[T7, Trahan, D.J....|
|[T7, Streelman, K...|
|[T11, Furyk, Jim,...|
|[T11, Lamely, Der...|
|[T11, Marino, Ste...|
|[T14, Love III, D...|
|[T14, Allenby, Ro...|
|[T14, Van Pelt, B...|
|[T17, Baird, Brin...|
|[T17, Choi, K.J.,...|
|[T17, Haas, Bill,...|
|[T17, Points, D.A...|
+--------------------+
only showing top 20 rows



1213

In [15]:
relevantCols = []
relevantCols.append("col." + label)
relevantCols.append("col." + meta)
for feat in features:
    relevantCols.append("col." + feat)
stats = allyearsstatscol.select(*relevantCols)
stats.show()

+-----+-----------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+
|total|      player_name|              sg_app|              sg_arg|              sg_ott|             sg_putt|             sg_t2g|          sg_total|
+-----+-----------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+
|  -11|       Els, Ernie|              1.8115| 0.30024999999999996|              0.6555|              1.0145|            2.76775|           3.78225|
|   -9|Molinari, Edoardo|              1.6615|              0.7035|              0.1255|              0.7915| 2.4909999999999997|3.2824999999999998|
|   -9|        Na, Kevin|  1.8159999999999998| 0.36649999999999994|-0.03424999999999999|               1.134|            2.14825|           3.28225|
|   -8|     Couch, Chris|             1.29975|            -0.61575|               0.907|  1.44099999999999

In [16]:
stats.count()

1213

### Now we have a dataset of 1200+ samples, and all the features we need. 

In [17]:
# import some ml goodness
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [19]:
# scale our data
vector_assembler = VectorAssembler(inputCols = features, outputCol = 'features')
feature_vector_df = vector_assembler.transform(stats)
feature_vector_df = feature_vector_df.select(['features', label])
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel =  scaler.fit(feature_vector_df.select("features"))
scaledData = scalerModel.transform(feature_vector_df)
# split into training and test sets
train, test = scaledData.randomSplit([0.9, 0.1], seed=343)
train.show()

+--------------------+-----+--------------------+
|            features|total|      scaledFeatures|
+--------------------+-----+--------------------+
|[-5.379,-0.8975,-...|   23|[0.0,0.3561596990...|
|[-4.9220000000000...|   10|[0.05517657712043...|
|[-4.3675,1.0215,-...|    5|[0.12212496226984...|
|[-4.312,-0.1275,0...|   10|[0.12882583760941...|
|[-4.1485,-0.02300...|    7|[0.14856625415031...|
|[-4.144,0.1925,-0...|   10|[0.14910956836703...|
|[-3.9765,-1.687,-...|   15|[0.16933293087835...|
|[-3.8919999999999...|   14|[0.17953516450347...|
|[-3.8259999999999...|   13|[0.18750377301539...|
|[-3.7904999999999...|   12|[0.19178991850286...|
|[-3.7335000000000...|   15|[0.19867189858134...|
|[-3.695,-1.563,0....|    7|[0.20332025354663...|
|[-3.5444999999999...|   18|[0.22149109568367...|
|[-3.4395,-0.516,-...|   13|[0.23416842740718...|
|[-3.3994999999999...|    9|[0.23899788711137...|
|[-3.3955,0.7015,-...|    8|[0.23948083308179...|
|[-3.351,0.2540000...|    9|[0.24485360700271...|


In [23]:
lr = LinearRegression(featuresCol = 'scaledFeatures', labelCol=label, maxIter=5)
lr_model = lr.fit(train)
print("Training RMSE: {}".format(lr_model.summary.rootMeanSquaredError))

Training RMSE: 2.85116525502


In [24]:
predictions = lr_model.transform(test).select("total", "prediction")
predictions.show()

+-----+-------------------+
|total|         prediction|
+-----+-------------------+
|   17|  20.05692685480244|
|    5|  8.647448302244811|
|   12| 12.554556094313739|
|    3| 2.9758025144183193|
|    5|  7.828870662049667|
|   15|  15.82178764702985|
|    8|   9.01755459888409|
|    7|  7.597294504526872|
|    9| 10.732052319679497|
|    5| 1.4451758417399105|
|    1|  2.294618061363508|
|   11|  3.990199363341162|
|    3|  3.657688301991584|
|    2| 2.1852338577391492|
|    9|  8.648645104816751|
|    3| 3.6802823122670816|
|    4|  6.251250938135854|
|    4| 2.3927815719071077|
|   -2|-2.7077875098912045|
|    9|  9.787426333447002|
+-----+-------------------+
only showing top 20 rows



In [25]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(predictions)
print("RMSE: %.3f" % rmse)

RMSE: 2.461


In [30]:
# stop the spark context at the end of execution
sc.stop()